In [1]:
import numpy as np
import pandas as pd
from tqdm import tqdm
from sklearn.svm import SVC
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler

from sklearn.metrics import confusion_matrix
from keras.layers import Dense, Activation, Dropout
from sklearn.metrics import classification_report
from keras.models import Sequential
from keras.utils import to_categorical
from sklearn.metrics import f1_score
import matplotlib.pyplot as plt
import seaborn as sns
import random

from imblearn.under_sampling import RandomUnderSampler
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import average_precision_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import balanced_accuracy_score, roc_auc_score
from imblearn.over_sampling import RandomOverSampler
from sklearn.model_selection import GridSearchCV

from tensorflow.keras import backend as K
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import balanced_accuracy_score, roc_auc_score
from sklearn.utils.class_weight import compute_class_weight
from sklearn.model_selection import LeaveOneGroupOut
import joblib

In [2]:
df1 = pd.read_csv('/Users/ohidabinteamin/Documents/Stress Prediction Project Three Datasets/StudentLife/week 01/PhoneLog/raw_phonelog_features_studentlife.csv')
df1 = df1.rename(columns={'Date': 'date'})
df1 = df1.drop('Unnamed: 0', axis=1)
df2 = pd.read_csv('/Users/ohidabinteamin/Documents/Stress Prediction Project Three Datasets/StudentLife/week 01/Stress/recreating_dailystress_features.csv')
df2 = df2.drop('Unnamed: 0', axis=1)

In [3]:
df = pd.merge(df1, df2, on=['uid', 'date'])
print(df.columns)

df = df.sort_values(by='date')

Index(['date', 'uid', 'morning_phonecharge_duration',
       'morning_phonecharge_frequency', 'morning_phonelock_duration',
       'morning_phonelock_frequency', 'morning_dark_duration',
       'afternoon_phonecharge_duration', 'afternoon_phonecharge_frequency',
       'afternoon_phonelock_duration', 'afternoon_phonelock_frequency',
       'afternoon_dark_duration', 'evening_phonecharge_duration',
       'evening_phonecharge_frequency', 'evening_phonelock_duration',
       'evening_phonelock_frequency', 'evening_dark_duration',
       'night_phonecharge_duration', 'night_phonecharge_frequency',
       'night_phonelock_duration', 'night_phonelock_frequency',
       'night_dark_duration', 'stress_ratings'],
      dtype='object')


In [4]:
df.isnull().sum()

date                                 0
uid                                  0
morning_phonecharge_duration       205
morning_phonecharge_frequency      205
morning_phonelock_duration           5
morning_phonelock_frequency          5
morning_dark_duration               47
afternoon_phonecharge_duration       0
afternoon_phonecharge_frequency      0
afternoon_phonelock_duration         0
afternoon_phonelock_frequency        0
afternoon_dark_duration              0
evening_phonecharge_duration         5
evening_phonecharge_frequency        5
evening_phonelock_duration           0
evening_phonelock_frequency          0
evening_dark_duration                0
night_phonecharge_duration           5
night_phonecharge_frequency          5
night_phonelock_duration             0
night_phonelock_frequency            0
night_dark_duration                  0
stress_ratings                       0
dtype: int64

In [5]:
df = df.dropna()
print(len(df))

3349


In [6]:
df.columns

Index(['date', 'uid', 'morning_phonecharge_duration',
       'morning_phonecharge_frequency', 'morning_phonelock_duration',
       'morning_phonelock_frequency', 'morning_dark_duration',
       'afternoon_phonecharge_duration', 'afternoon_phonecharge_frequency',
       'afternoon_phonelock_duration', 'afternoon_phonelock_frequency',
       'afternoon_dark_duration', 'evening_phonecharge_duration',
       'evening_phonecharge_frequency', 'evening_phonelock_duration',
       'evening_phonelock_frequency', 'evening_dark_duration',
       'night_phonecharge_duration', 'night_phonecharge_frequency',
       'night_phonelock_duration', 'night_phonelock_frequency',
       'night_dark_duration', 'stress_ratings'],
      dtype='object')

In [7]:
df['stress_ratings'].value_counts()

stress_ratings
medium stress    1328
high stress      1043
low stress        978
Name: count, dtype: int64

In [8]:
binary_lh_data = df[df['stress_ratings'].isin(['low stress', 'high stress'])]

In [9]:
X = binary_lh_data.drop(columns=['stress_ratings', 'uid', 'date'])
y = binary_lh_data['stress_ratings']
groups = binary_lh_data['uid']

stress_map = {'low stress': 0, 'high stress': 1}
y_encoded = y.map(stress_map).values 

In [10]:
logo = LeaveOneGroupOut()

best_thresholds = []
balanced_accs = []
auc_scores = []

In [11]:
seed_value = 42
np.random.seed(seed_value)
random.seed(seed_value)
tf.random.set_seed(seed_value)

In [12]:
num_splits = len(np.unique(groups))
logo = LeaveOneGroupOut()

In [13]:
num_splits = len(np.unique(groups))
class_weights = compute_class_weight('balanced', classes=np.unique(y_encoded), y=y_encoded)
class_weight_dict = dict(enumerate(class_weights))

param_grid = {
    'C': [0.1, 1, 10, 100],
    'gamma': ['scale', 'auto', 0.1, 0.01, 0.001],
    'class_weight': [class_weight_dict]
}

best_thresholds = []
balanced_accs = []
auc_scores = []
auprc_scores = []
best_params_list = []

In [14]:
with tqdm(total=num_splits, desc="LOSO CV Progress", unit="fold", bar_format="{l_bar}{bar} | {n_fmt}/{total_fmt} [{elapsed}<{remaining}] {percentage:3.0f}%") as pbar:
    for train_idx, test_idx in logo.split(X, y_encoded, groups=groups):
        X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
        y_train, y_test = y_encoded[train_idx], y_encoded[test_idx]  

        print(f"Train shape: {X_train.shape}, {y_train.shape}")

        scaler = StandardScaler()
        X_train_normalized = scaler.fit_transform(X_train)
        X_test_normalized = scaler.transform(X_test)

        print(f"Normalized shapes: Train {X_train_normalized.shape}, Test {X_test_normalized.shape}")

        print('Performing Grid Search for SVM')
        grid_search = GridSearchCV(
            SVC(kernel='rbf', probability=True),
            param_grid,
            cv=5,
            scoring='balanced_accuracy',
            n_jobs=-1 
        )

        grid_search.fit(X_train_normalized, y_train)
        model_svm_rbf = grid_search.best_estimator_
        
        print("Best parameters:", grid_search.best_params_)
        print("Best cross-validation score:", grid_search.best_score_)
        best_params_list.append(grid_search.best_params_)

        y_test_pred_proba = model_svm_rbf.predict_proba(X_test_normalized)[:, 1]  
        thresholds = np.arange(0.01, 1.0, 0.01)
        best_threshold = max(thresholds, key=lambda t: balanced_accuracy_score(y_test, (y_test_pred_proba > t).astype(int)))

        if len(np.unique(y_test)) > 1:
            auc_score = roc_auc_score(y_test, y_test_pred_proba)
            auprc = average_precision_score(y_test, y_test_pred_proba)
            auc_scores.append(auc_score)
            auprc_scores.append(auprc)
            print(f"AUC Score: {auc_score}")
            print(f"AUPRC Score: {auprc}")
        else:
            auc_scores.append(None)
            auprc_scores.append(None)
            print(f"Skipping AUC and AUPRC computation for this fold as y_test contains only one class: {np.unique(y_test)}")

        y_test_pred_binary = (y_test_pred_proba > best_threshold).astype(int)
        balanced_acc = balanced_accuracy_score(y_test, y_test_pred_binary)
        balanced_accs.append(balanced_acc)
        best_thresholds.append(best_threshold)
        
        print(f"Balanced Accuracy: {balanced_acc}")
        print(f"Best Threshold: {best_threshold}")

        pbar.update(1)

LOSO CV Progress:   0%|                                    | 0/43 [00:00<?]   0%

Train shape: (1950, 20), (1950,)
Normalized shapes: Train (1950, 20), Test (71, 20)
Performing Grid Search for SVM


LOSO CV Progress:   2%|▋                               | 1/43 [00:06<04:35]   2%

Best parameters: {'C': 10, 'class_weight': {0: 1.0332310838445808, 1: 0.9688398849472675}, 'gamma': 'scale'}
Best cross-validation score: 0.5339768388633275
AUC Score: 0.4292682926829268
AUPRC Score: 0.5889641277402399
Balanced Accuracy: 0.5308943089430894
Best Threshold: 0.53
Train shape: (1978, 20), (1978,)
Normalized shapes: Train (1978, 20), Test (43, 20)
Performing Grid Search for SVM


LOSO CV Progress:   5%|█▍                              | 2/43 [00:12<04:15]   5%

...

Best parameters: {'C': 10, 'class_weight': {0: 1.0332310838445808, 1: 0.9688398849472675}, 'gamma': 'scale'}
Best cross-validation score: 0.5271392043507392
AUC Score: 0.6061007957559681
AUPRC Score: 0.7788175603413212
Balanced Accuracy: 0.6147214854111406
Best Threshold: 0.51
Train shape: (1939, 20), (1939,)
Normalized shapes: Train (1939, 20), Test (82, 20)
Performing Grid Search for SVM


LOSO CV Progress:  98%|█████████████████████████████▎ | 42/43 [04:34<00:06]  98%

Best parameters: {'C': 10, 'class_weight': {0: 1.0332310838445808, 1: 0.9688398849472675}, 'gamma': 'scale'}
Best cross-validation score: 0.5300475227954873
AUC Score: 0.5909502262443439
AUPRC Score: 0.8353570540731476
Balanced Accuracy: 0.5900452488687783
Best Threshold: 0.5
Train shape: (1869, 20), (1869,)
Normalized shapes: Train (1869, 20), Test (152, 20)
Performing Grid Search for SVM


LOSO CV Progress: 100%|██████████████████████████████ | 43/43 [04:39<00:00] 100%

Best parameters: {'C': 10, 'class_weight': {0: 1.0332310838445808, 1: 0.9688398849472675}, 'gamma': 0.1}
Best cross-validation score: 0.5162435185520711
AUC Score: 0.5486979166666667
AUPRC Score: 0.25565671918497423
Balanced Accuracy: 0.571875
Best Threshold: 0.53


In [15]:
print('SVM with Radial basis function (RBF) kernel')
print('Median, 25th Percentile, 75th Percentile: ')

if auc_scores:
    auc_scores_valid = [score for score in auc_scores if score is not None] 

    auc_median = np.median(auc_scores_valid)
    auc_25_percentile = np.percentile(auc_scores_valid, 25)
    auc_75_percentile = np.percentile(auc_scores_valid, 75)
    print(f"AUC Score - Median: {auc_median}, 25th Percentile: {auc_25_percentile}, 75th Percentile: {auc_75_percentile}")

if auprc_scores:
    auprc_scores_valid = [score for score in auprc_scores if score is not None] 

    auprc_median = np.median(auprc_scores_valid)
    auprc_25_percentile = np.percentile(auprc_scores_valid, 25)
    auprc_75_percentile = np.percentile(auprc_scores_valid, 75)
    print(f"AUPRC Score - Median: {auprc_median}, 25th Percentile: {auprc_25_percentile}, 75th Percentile: {auprc_75_percentile}")

balanced_acc_median = np.median(balanced_accs)
balanced_acc_25_percentile = np.percentile(balanced_accs, 25)
balanced_acc_75_percentile = np.percentile(balanced_accs, 75)

print(f"Balanced Accuracy - Median: {balanced_acc_median}, 25th Percentile: {balanced_acc_25_percentile}, 75th Percentile: {balanced_acc_75_percentile}")

SVM with Radial basis function (RBF) kernel
Median, 25th Percentile, 75th Percentile: 
AUC Score - Median: 0.4987212276214834, 25th Percentile: 0.43872341426689254, 75th Percentile: 0.6032640730916592
AUPRC Score - Median: 0.5961669880042052, 25th Percentile: 0.4641125969668354, 75th Percentile: 0.7841502338806707
Balanced Accuracy - Median: 0.5625, 25th Percentile: 0.5299974190218093, 75th Percentile: 0.6292635658914729
